In [ ]:
import sys
import os

notebook_path = os.getcwd() 
parent_dir = os.path.dirname(notebook_path)
project_root = os.path.dirname(parent_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [3]:
import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [ ]:
import torch
from math import pi

state_dim_nl = 2
obs_dim_nl = 2

alpha_true = 0.9
beta_true = 1.1
phi_true = 0.1 * pi
delta_true = 0.01
a_true = 1.0
b_true = 1.0
c_true = 0.0

f_true_nonlinear = lambda x: alpha_true * torch.sin(beta_true * x + phi_true) + delta_true
h_true_nonlinear = lambda x: a_true * (b_true * x + c_true)**2

Q_true_nl = torch.eye(state_dim_nl) * 0.5
R_true_nl = torch.eye(obs_dim_nl) * 0.1
Ex0_true_nl = torch.tensor([[1.0], [0.0]])
P0_true_nl = torch.eye(state_dim_nl) * 1.5

alpha_model = 1.0
beta_model = 1.0
phi_model = 0.0
delta_model = 0.0
a_model = 1.0
b_model = 1.0
c_model = 0.0


f_model_nonlinear = lambda x: alpha_model * torch.sin(beta_model * x + phi_model) + delta_model
h_model_nonlinear = lambda x: a_model * (b_model * x + c_model)**2


Q_model_nl = torch.eye(state_dim_nl) * 0.1
R_model_nl = R_true_nl 
Ex0_model_nl = torch.tensor([[0.5], [0.5]])
P0_model_nl = torch.eye(state_dim_nl) * 1.0


print("\nInicializuji 2D 'Synthetic' nelineární systém...")

sys_true = Systems.DynamicSystem(
    state_dim=state_dim_nl, obs_dim=obs_dim_nl,
    Ex0=Ex0_true_nl, P0=P0_true_nl,
    Q=Q_true_nl, R=R_true_nl,
    f=f_true_nonlinear, h=h_true_nonlinear,
    device=device
)

sys_model = Systems.DynamicSystem(
    state_dim=state_dim_nl, obs_dim=obs_dim_nl,
    Ex0=Ex0_model_nl, P0=P0_model_nl,
    Q=Q_model_nl, R=R_model_nl,
    f=f_model_nonlinear, h=h_model_nonlinear,
    device=device
)

print("... Nelineární systém inicializován.")


Inicializuji 2D 'Synthetic' nelineární systém...
... Nelineární systém inicializován.


In [ ]:
TRAIN_SEQ_LEN = 100  
VALID_SEQ_LEN = 200  
TEST_SEQ_LEN = 1000 

NUM_TRAIN_TRAJ = 50 
NUM_VALID_TRAJ = 20 
NUM_TEST_TRAJ = 100

BATCH_SIZE = 256

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
from state_NN_models import StateKalmanNet_v2

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knet = StateKalmanNet_v2(sys_model, device=device).to(device)
trainer.train_state_KalmanNet_sliding_window(
    model=state_knet, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=200, 
    lr=1e-3,
    early_stopping_patience=20,
    tbptt_k=2,
    tbptt_w=8
)

Inicializuji váhy pro StateKalmanNet_v2 (Arch 1)...


/home/luky/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


INFO: Detekováno z atributu modelu, že vrací kovarianci: False
INFO: Spouštím trénink s TBPTT(k=2, w=8)
Nové nejlepší model uloženo! Epoch [1/200], Train Loss: 1.179423, Val Loss: 1.033633
Nové nejlepší model uloženo! Epoch [2/200], Train Loss: 0.814482, Val Loss: 0.729515
Nové nejlepší model uloženo! Epoch [3/200], Train Loss: 0.732644, Val Loss: 0.706181
Nové nejlepší model uloženo! Epoch [4/200], Train Loss: 0.722937, Val Loss: 0.679229
Epoch [5/200], Train Loss: 0.725678, Val Loss: 0.691842
Epoch [10/200], Train Loss: 0.702758, Val Loss: 0.666757
Nové nejlepší model uloženo! Epoch [10/200], Train Loss: 0.702758, Val Loss: 0.666757
Epoch [15/200], Train Loss: 0.642699, Val Loss: 0.683181
Epoch [20/200], Train Loss: 0.628834, Val Loss: 0.680452
Epoch [25/200], Train Loss: 0.610711, Val Loss: 0.681731
Epoch [30/200], Train Loss: 0.605822, Val Loss: 0.677716

Early stopping spuštěno po 30 epochách.
Trénování dokončeno.
Načítám nejlepší model s validační chybou: 0.666757


StateKalmanNet_v2(
  (dnn): DNN_KalmanNet_v2(
    (input_layer): Sequential(
      (0): Linear(in_features=8, out_features=320, bias=True)
      (1): ReLU()
    )
    (gru): GRU(320, 80)
    (output_hidden_layer): Sequential(
      (0): Linear(in_features=80, out_features=16, bias=True)
      (1): ReLU()
    )
    (output_final_linear): Linear(in_features=16, out_features=4, bias=True)
  )
)

In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
from state_NN_models import StateKalmanNetWithKnownR

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knetR = StateKalmanNetWithKnownR(sys_model, device=device, hidden_size_multiplier=12).to(device)
trainer.train_state_KalmanNet(
    model=state_knetR, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=200, 
    lr=1e-3,
    early_stopping_patience=20
)

INFO: Detekováno z atributu modelu, že vrací kovarianci: True
Nové nejlepší model uloženo! Epoch [1/200], Train Loss: 0.916056, Val Loss: 0.732679
Epoch [5/200], Train Loss: 0.803721, Val Loss: 0.762531, Avg Cov Trace: 0.044729
Nové nejlepší model uloženo! Epoch [9/200], Train Loss: 0.786449, Val Loss: 0.731789
Epoch [10/200], Train Loss: 0.772787, Val Loss: 0.720548, Avg Cov Trace: 0.061747
Nové nejlepší model uloženo! Epoch [10/200], Train Loss: 0.772787, Val Loss: 0.720548
Nové nejlepší model uloženo! Epoch [11/200], Train Loss: 0.758176, Val Loss: 0.710557
Nové nejlepší model uloženo! Epoch [12/200], Train Loss: 0.745505, Val Loss: 0.702113
Nové nejlepší model uloženo! Epoch [13/200], Train Loss: 0.734762, Val Loss: 0.695783
Nové nejlepší model uloženo! Epoch [14/200], Train Loss: 0.726243, Val Loss: 0.691572
Epoch [15/200], Train Loss: 0.720326, Val Loss: 0.688509, Avg Cov Trace: 0.075863
Nové nejlepší model uloženo! Epoch [15/200], Train Loss: 0.720326, Val Loss: 0.688509
Nové ne

StateKalmanNetWithKnownR(
  (dnn): DNN_KalmanNet(
    (input_layer): Linear(in_features=4, out_features=96, bias=True)
    (gru): GRU(96, 96)
    (output_layer): Linear(in_features=96, out_features=4, bias=True)
  )
)

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
try:
    trained_model_classic = state_knet
    trained_model_knetR = state_knetR
    print("INFO: Všechny natrénované modely nalezeny a přiřazeny.")
except NameError:
    print("VAROVÁNÍ: Některé z proměnných `trained_model`, `state_knet`, nebo `state_knetR` nebyly nalezeny.")


TEST_SEQ_LEN = 3000
NUM_TEST_TRAJ = 10

print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
ukf_ideal = Filters.UnscentedKalmanFilter(sys_true)
print("Všechny model-based filtry (EKF, UKF, AEKF) inicializovány.")

all_x_true_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
all_x_hat_classic_knet_cpu = []
all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []
all_x_hat_ukf_mismatched_cpu, all_P_hat_ukf_mismatched_cpu = [], []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
all_x_hat_aekf_mismatched_cpu, all_P_hat_aekf_mismatched_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")
 
trained_model_classic.eval()
trained_model_knetR.eval()

with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
        trained_model_classic.reset(batch_size=1, initial_state=initial_state)
        classic_knet_preds = []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t = trained_model_classic.step(y_test_seq_gpu[t, :].unsqueeze(0))
            classic_knet_preds.append(x_filtered_t)
        full_x_hat_classic_knet = torch.cat([initial_state, torch.cat(classic_knet_preds, dim=0)], dim=0)
        
        trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
        knetR_preds_x, knetR_preds_P = [], []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t, P_filtered_t = trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
            knetR_preds_x.append(x_filtered_t)
            knetR_preds_P.append(P_filtered_t)
        full_x_hat_knetR = torch.cat([initial_state, torch.cat(knetR_preds_x, dim=0)], dim=0)
        processed_P_list = []
        for p_tensor in knetR_preds_P:

            while p_tensor.dim() < 2:
                p_tensor = p_tensor.unsqueeze(-1)

            if p_tensor.dim() > 2 and p_tensor.shape[0] == 1:
                p_tensor = p_tensor.squeeze(0)
            processed_P_list.append(p_tensor)


        P_sequence_knetR = torch.stack(processed_P_list, dim=0)
        

        P0_for_cat = sys_model.P0.clone()
        while P0_for_cat.dim() < P_sequence_knetR.dim():
            P0_for_cat = P0_for_cat.unsqueeze(0)
            
        full_P_hat_knetR = torch.cat([P0_for_cat, P_sequence_knetR], dim=0)

        ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)

        full_x_hat_ekf_m = ekf_m_res['x_filtered'] 
        full_P_hat_ekf_m = ekf_m_res['P_filtered'] 

        ekf_i_res = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        full_x_hat_ekf_i = ekf_i_res['x_filtered']
        full_P_hat_ekf_i = ekf_i_res['P_filtered']

        ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)

        full_x_hat_ukf_m = ukf_m_res['x_filtered']
        full_P_hat_ukf_m = ukf_m_res['P_filtered']

        ukf_i_res = ukf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)

        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet.cpu())
        all_x_hat_knetR_cpu.append(full_x_hat_knetR.cpu()); all_P_hat_knetR_cpu.append(full_P_hat_knetR.cpu())
        all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_m.cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_i.cpu()); all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_i.cpu())
        all_x_hat_ukf_mismatched_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_mismatched_cpu.append(full_P_hat_ukf_m.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

mse_bkn, anees_bkn = [], []; mse_classic_knet = []; mse_knetR, anees_knetR = [], []
mse_ekf_mis, anees_ekf_mis = [], []; mse_ekf_ideal, anees_ekf_ideal = [], []
mse_ukf_mis, anees_ukf_mis = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []
mse_aekf_mis, anees_aekf_mis = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        def get_metrics(x_hat, P_hat):
            mse = F.mse_loss(x_hat[1:], x_true[1:]).item()
            anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat.unsqueeze(0), P_hat.unsqueeze(0))
            return mse, anees

        mse = F.mse_loss(all_x_hat_classic_knet_cpu[i][1:], x_true[1:]).item(); mse_classic_knet.append(mse)
        mse, anees = get_metrics(all_x_hat_knetR_cpu[i], all_P_hat_knetR_cpu[i]); mse_knetR.append(mse); anees_knetR.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_mismatched_cpu[i], all_P_hat_ekf_mismatched_cpu[i]); mse_ekf_mis.append(mse); anees_ekf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_ideal_cpu[i], all_P_hat_ekf_ideal_cpu[i]); mse_ekf_ideal.append(mse); anees_ekf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_mismatched_cpu[i], all_P_hat_ukf_mismatched_cpu[i]); mse_ukf_mis.append(mse); anees_ukf_mis.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)

def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 80)
print(f"{'--- Data-Driven Models ---':<35} | {'(nižší je lepší)':<20} | {'(bližší ' + str(float(state_dim_for_nees)) + ' je lepší)':<20}")
print(f"{'KNet (pouze MSE)':<35} | {avg(mse_classic_knet):<20.4f} | {'N/A':<20}")
print(f"{'KNet with Known R (KNetR)':<35} | {avg(mse_knetR):<20.4f} | {avg(anees_knetR):<20.4f}")
print("-" * 80)
print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_mis):<20.4f} | {avg(anees_ekf_mis):<20.4f}")
print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_mis):<20.4f} | {avg(anees_ukf_mis):<20.4f}")
print("-" * 80)
print(f"{'--- Benchmarks ---':<35} | {'':<20} | {'':<20}")
print(f"{'EKF (Ideální model)':<35} | {avg(mse_ekf_ideal):<20.4f} | {avg(anees_ekf_ideal):<20.4f}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
print("="*80)

INFO: Všechny natrénované modely nalezeny a přiřazeny.

Generuji 10 testovacích trajektorií o délce 3000...
Generování dat dokončeno.
Všechny model-based filtry (EKF, UKF, AEKF) inicializovány.

Vyhodnocuji modely na 10 testovacích trajektoriích...
Dokončena trajektorie 1/10...
Dokončena trajektorie 2/10...
Dokončena trajektorie 3/10...
Dokončena trajektorie 4/10...
Dokončena trajektorie 5/10...
Dokončena trajektorie 6/10...
Dokončena trajektorie 7/10...
Dokončena trajektorie 8/10...
Dokončena trajektorie 9/10...
Dokončena trajektorie 10/10...

Počítám finální metriky pro jednotlivé trajektorie...

FINÁLNÍ VÝSLEDKY (průměr přes 10 běhů)
Model                               | Průměrné MSE         | Průměrný ANEES      
--------------------------------------------------------------------------------
--- Data-Driven Models ---          | (nižší je lepší)     | (bližší 2.0 je lepší)
KNet (pouze MSE)                    | 0.6830               | N/A                 
KNet with Known R (KNetR)  